<a href="https://colab.research.google.com/github/yoojinleee/CarGame/blob/main/CarGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pygame
import sys

# Initializing the game
pygame.init()

# Making the settings for the screen display
screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Car Racing Game")

# Frame rate for moving car
clock = pygame.time.Clock()

# Uploading an image of the car
car_image = pygame.image.load("car.png") # Note to teammates: I'm having trouble figuring out how to add the image of the car into our code
car_rect = car_image.get_rect(center=(400, 500)) # Can resize later depending

# Making the car move
car_speed = 5

# Trying to code the ability to control the movement of the car
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        end_time = pygame.time.get_ticks()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                car_rect.x -= car_speed
            elif event.key == pygame.K_RIGHT:
                car_rect.x += car_speed


FileNotFoundError: No file 'car.png' found in working directory '/content'.